<a href="https://colab.research.google.com/github/lfslucius/Members-Subscription-Solver/blob/main/Applied_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [ ]:
# read data for preparation
meninfo = pd.read_excel(open('Assignments_Data1.xlsx','rb'), sheet_name='Mem_info')
iteminfo = pd.read_excel(open('Assignments_Data1.xlsx','rb'), sheet_name='Item_info')
delinfo = pd.read_excel(open('Assignments_Data1.xlsx','rb'), sheet_name='Delivery_cost')

In [ ]:
# transfter item data into lists
itemid = iteminfo['item_id'].to_list()
inventory = iteminfo['Inventory'].to_list()
ScoreAgeSeg = iteminfo.loc[:, 'Score_Age_Seg_1':'Score_Age_Seg_5']
ScoreAgeSeg = ScoreAgeSeg.T
ScoreRegSeg = iteminfo.loc[:, 'Score_Reg_Seg_1':'Score_Reg_Seg_7']
ScoreRegSeg = ScoreRegSeg.T

In [ ]:
# transfter cust data into lists
memid = meninfo['member_id'].to_list()
AgeSeg = meninfo['age_seg'].to_list()
RegSeg = meninfo['region_seg'].to_list()
item_hist = meninfo.loc[:,'item1':'item9'].values.tolist()

In [ ]:
# transfter delivery cost data into lists
regionid = delinfo['region_seg'].to_list()
del_hist = delinfo.loc[:,'item1':'item9'].values.tolist()

In [ ]:
# Rearrange the format of ScoreAgeSeg in terms of members' age segments
ScoreAgeSegList = ScoreAgeSeg.values.tolist()
MemAgeScore = []
for K in AgeSeg:
    MemAgeScore.append(ScoreAgeSegList[K-1])
MemAgeScore

[[7.21, 6.75, 7.18, 6.97, 7.36, 7.32, 7.41, 7.61, 7.63],
 [7.21, 6.75, 7.18, 6.97, 7.36, 7.32, 7.41, 7.61, 7.63],
 [7.19, 7.05, 6.82, 7.1, 8.16, 7.64, 7.29, 7.09, 6.25],
 [7.18, 9.0, 6.27, 7.08, 7.71, 8.0, 6.79, 7.26, 8.5],
 [7.21, 6.75, 7.18, 6.97, 7.36, 7.32, 7.41, 7.61, 7.63],
 [7.21, 6.75, 7.18, 6.97, 7.36, 7.32, 7.41, 7.61, 7.63],
 [7.5, 6.5, 6.88, 8.0, 7.21, 7.5, 8.0, 8.0, 5.0],
 [7.19, 7.05, 6.82, 7.1, 8.16, 7.64, 7.29, 7.09, 6.25],
 [7.19, 7.05, 6.82, 7.1, 8.16, 7.64, 7.29, 7.09, 6.25],
 [7.21, 6.75, 7.18, 6.97, 7.36, 7.32, 7.41, 7.61, 7.63],
 [7.19, 7.05, 6.82, 7.1, 8.16, 7.64, 7.29, 7.09, 6.25],
 [7.21, 6.75, 7.18, 6.97, 7.36, 7.32, 7.41, 7.61, 7.63],
 [7.19, 7.05, 6.82, 7.1, 8.16, 7.64, 7.29, 7.09, 6.25],
 [7.21, 6.75, 7.18, 6.97, 7.36, 7.32, 7.41, 7.61, 7.63],
 [7.18, 9.0, 6.27, 7.08, 7.71, 8.0, 6.79, 7.26, 8.5],
 [7.18, 9.0, 6.27, 7.08, 7.71, 8.0, 6.79, 7.26, 8.5],
 [7.18, 9.0, 6.27, 7.08, 7.71, 8.0, 6.79, 7.26, 8.5],
 [7.19, 7.05, 6.82, 7.1, 8.16, 7.64, 7.29, 7.09, 6.25],

In [ ]:
# Rearrange the format of ScoreRegSeg in terms of members' region segments
ScoreRegSegList = ScoreRegSeg.values.tolist()
MemRegScore = []
for K in RegSeg:
    MemRegScore.append(ScoreRegSegList[K-1])
MemRegScore

[[8.5, 7.19, 6.88, 7.5, 7.57, 7.08, 7.63, 7.75, 8.0],
 [7.4, 7.36, 6.78, 7.27, 7.94, 7.75, 7.11, 7.32, 6.17],
 [7.79, 7.33, 6.5, 7.63, 7.0, 7.75, 7.5, 7.4, 8.0],
 [7.31, 6.93, 7.6, 7.0, 7.0, 8.0, 6.33, 7.17, 7.0],
 [7.31, 6.93, 7.6, 7.0, 7.0, 8.0, 6.33, 7.17, 7.0],
 [7.4, 7.36, 6.78, 7.27, 7.94, 7.75, 7.11, 7.32, 6.17],
 [7.31, 6.93, 7.6, 7.0, 7.0, 8.0, 6.33, 7.17, 7.0],
 [7.79, 7.33, 6.5, 7.63, 7.0, 7.75, 7.5, 7.4, 8.0],
 [8.33, 6.17, 7.07, 6.67, 7.17, 7.44, 7.0, 7.05, 6.4],
 [7.4, 7.36, 6.78, 7.27, 7.94, 7.75, 7.11, 7.32, 6.17],
 [7.8, 7.47, 7.2, 4.67, 8.0, 6.36, 8.56, 6.0, 9.0],
 [8.33, 6.17, 7.07, 6.67, 7.17, 7.44, 7.0, 7.05, 6.4],
 [7.4, 7.36, 6.78, 7.27, 7.94, 7.75, 7.11, 7.32, 6.17],
 [7.8, 7.47, 7.2, 4.67, 8.0, 6.36, 8.56, 6.0, 9.0],
 [7.4, 7.36, 6.78, 7.27, 7.94, 7.75, 7.11, 7.32, 6.17],
 [7.96, 6.38, 6.61, 6.92, 8.3, 7.68, 7.15, 7.5, 8.36],
 [7.4, 7.36, 6.78, 7.27, 7.94, 7.75, 7.11, 7.32, 6.17],
 [7.4, 7.36, 6.78, 7.27, 7.94, 7.75, 7.11, 7.32, 6.17],
 [7.4, 7.36, 6.78, 7.27, 

In [ ]:
# Rearrange the format of deliver costs in terms of members' region segments
MemDelCost = []
for K in RegSeg:
    MemDelCost.append(del_hist[K-1])
MemDelCost

[[10, 9, 12, 12, 12, 13, 10, 12, 12],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [10, 9, 12, 12, 12, 13, 10, 12, 12],
 [10, 9, 12, 12, 12, 13, 10, 12, 12],
 [10, 9, 12, 12, 12, 13, 10, 12, 12],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [10, 9, 12, 12, 12, 13, 10, 12, 12],
 [10, 9, 12, 12, 12, 13, 10, 12, 12],
 [12, 11, 13, 13, 13, 14, 12, 13, 13],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [12, 11, 13, 13, 13, 14, 12, 13, 13],
 [12, 11, 13, 13, 13, 14, 12, 13, 13],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [12, 11, 13, 13, 13, 14, 12, 13, 13],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [17, 15, 18, 18, 18, 19, 17, 18, 18],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [12, 11, 13, 13, 13, 14, 12, 13, 13],
 [12, 11, 13, 13, 13, 14, 12, 13, 13],
 [10, 9, 12, 12, 12, 13, 10, 12, 12],
 [10, 9, 12, 12, 12, 13, 10, 12, 12],
 [12, 11, 13, 13, 13, 14, 12, 13, 13],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [8, 7, 9, 9, 9, 10, 8, 9, 9],
 [12, 11, 13

In [ ]:
#Declare a concrete model
model = ConcreteModel()
num_item = len(itemid) #number of items index them by i
num_mem = len(memid) #number of members index them by j
num_region = len(regionid) # number of regions index them by z
budget = 30000

#declare the decision variables
model.x = Var(range(num_item), range(num_mem), domain = Binary) #which 1 products are selected for each customer? index by [i,j]

#Constraint: each customer should receive 1 items
model.ItemNum = ConstraintList()
for j in range(num_mem):
    model.ItemNum.add(expr=sum(model.x[i,j] for i in range(num_item)) == 1)

#Constraint: the sum of items should be within the inventory
model.MaxNum = ConstraintList()
for i in range(num_item):
    model.MaxNum.add(expr=sum(model.x[i,j] for j in range(num_mem)) <= inventory[i])
    
#Constraint: each customer should not be sent with the products which he/she has received before
model.ItemUni = ConstraintList()
for j in range(num_mem):
    for i in range(num_item):
        model.ItemUni.add(expr=item_hist[j][i]+model.x[i,j] <= 1)
        
#Constraint: the total cost of delivering should be within the budget
model.DelCost = Constraint(expr=sum(model.x[i,j]*MemDelCost[j][i] for i in range(num_item) for j in range(num_mem)) <=20000)


#Objective1: Maximize the total average satisfaction rate of members to the items received  
sumscores = sum(model.x[i,j] * (MemAgeScore[j][i] + MemRegScore[j][i])/2 for i in range(num_item) for j in range(num_mem))
totalcount = num_mem
Average_Satisfaction_Rate = sumscores/totalcount
model.Objective = Objective(expr = Average_Satisfaction_Rate, sense = maximize)

In [ ]:
opt = SolverFactory("glpk")
opt.options['mipgap'] = .001
results = opt.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --mipgap 0.001 --write /var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmp_6by4ifo.glpk.raw
 --wglp /var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpv95_fwtx.glpk.glp
 --cpxlp /var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmp79p68juc.pyomo.lp
Reading problem data from '/var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmp79p68juc.pyomo.lp'...
/var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmp79p68juc.pyomo.lp:136963: warning: lower bound of variable 'x1' redefined
/var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmp79p68juc.pyomo.lp:136963: warning: upper bound of variable 'x1' redefined
16311 rows, 14671 columns, 58681 non-zeros
14670 integer variables, all of which are binary
151633 lines were read
Writing problem data to '/var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpv95_fwtx.glpk.glp'...
120648 lines were written
GLPK Integer Optimizer 5.0
16311 rows, 14671 columns, 58681 non-zeros
14670 integ

In [ ]:
#printing solutions
print("obj: Maximum Satisfaction Rate", model.Objective())

#printing costs
delcost = 0
for j in range(num_mem):
    for i in range(num_item):
        if model.x[i,j]() == 1:
            delcost=delcost+MemDelCost[j][i]
print("delivery costs:", delcost)

#printing plan
Plan1 = []
for j in range(num_mem):
    for i in range(num_item):
        if model.x[i,j]() == 1:
            Plan1.append(i+1)
    #print('Send Member', j+1, 'Products', Plan1)
    #Plan1 = [] 

data = {'member_id': memid,
        'Birch_Plan_Items_List': Plan1}
outputs1 = DataFrame(data)
outputs1.loc[0, 'Maximum Satisfaction Rate'] = model.Objective()
outputs1.loc[0, 'delivery costs'] = delcost

obj: Maximum Satisfaction Rate 7.838279141104266
delivery costs: 19174


In [ ]:
#Change the objective to minimize the cost while the satisfaction rate is keeping above 7
#Declare a concrete model
model2 = ConcreteModel()
num_item = len(itemid) #number of items index them by i
num_mem = len(memid) #number of members index them by j
num_region = len(regionid) # number of regions index them by z
budget = 30000

#declare the decision variables
model2.x = Var(range(num_item), range(num_mem), domain = Binary) #which 1 products are selected for each customer? index by [i,j]

#Constraint: each customer should receive 1 items
model2.ItemNum = ConstraintList()
for j in range(num_mem):
    model2.ItemNum.add(expr=sum(model2.x[i,j] for i in range(num_item)) == 1)

#Constraint: the sum of items should be within the inventory
model2.MaxNum = ConstraintList()
for i in range(num_item):
    model2.MaxNum.add(expr=sum(model2.x[i,j] for j in range(num_mem)) <= inventory[i])
    
#Constraint: each customer should not be sent with the products which he/she has received before
model2.ItemUni = ConstraintList()
for j in range(num_mem):
    for i in range(num_item):
        model2.ItemUni.add(expr=item_hist[j][i]+model2.x[i,j] <= 1)
        
#Constraint: keep the satisfaction rate above 7
sumscores = sum(model2.x[i,j] * (MemAgeScore[j][i] + MemRegScore[j][i])/2 for i in range(num_item) for j in range(num_mem))
totalcount = num_mem
model2.Scores= Constraint(expr= sumscores/totalcount >= 7)


#Objective2: Minimize the total delivery cost  
DelCost = sum(model2.x[i,j]*MemDelCost[j][i] for i in range(num_item) for j in range(num_mem))
model2.Objective = Objective(expr = DelCost, sense = minimize)

In [ ]:
opt = SolverFactory("glpk")
opt.options['mipgap'] = .001
results = opt.solve(model2, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --mipgap 0.001 --write /var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpi3iq0122.glpk.raw
 --wglp /var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpubo_dzel.glpk.glp
 --cpxlp /var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpavy3zl0s.pyomo.lp
Reading problem data from '/var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpavy3zl0s.pyomo.lp'...
/var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpavy3zl0s.pyomo.lp:136963: warning: lower bound of variable 'x1' redefined
/var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpavy3zl0s.pyomo.lp:136963: warning: upper bound of variable 'x1' redefined
16311 rows, 14671 columns, 58681 non-zeros
14670 integer variables, all of which are binary
151633 lines were read
Writing problem data to '/var/folders/nq/w9lsxdj51xqbbknjljnwnv7c0000gn/T/tmpubo_dzel.glpk.glp'...
120648 lines were written
GLPK Integer Optimizer 5.0
16311 rows, 14671 columns, 58681 non-zeros
14670 integ

In [ ]:
#printing solutions
print("obj: Minimized Cost", model2.Objective())

#print total average score
Tmsr= 0
for j in range(num_mem):
    for i in range(num_item):
        if model2.x[i,j]() == 1: 
            Tmsr=Tmsr+(MemAgeScore[j][i] + MemRegScore[j][i])/2
print("Total Average Satisfaction Rate:", Tmsr/num_mem)


#printing plan
Plan2 = []
for j in range(num_mem):
    for i in range(num_item):
        if model2.x[i,j]() == 1:
            Plan2.append(i+1)
    #print('Send Member', j+1, 'Products', Plan2)
    #Pla2n = []

data = {'member_id': memid,
        'Birch_Plan_Items_List': Plan2}
outputs2 = DataFrame(data)
outputs2.loc[0, 'Minimized Cost'] = model2.Objective()
outputs2.loc[0, 'Total Average Satisfaction Rate'] = Tmsr/num_mem

obj: Minimized Cost 18121.0
Total Average Satisfaction Rate: 7.153358895705525


In [ ]:
#export to csv
outputs1.to_csv('outputs1.csv', index=False)
outputs2.to_csv('outputs2.csv', index=False)